In [1]:
# Required Python Packages
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import datasets, linear_model


In [2]:
# Here we have to load the given car Train Data into dataset and test data into testdata using read
dataset = pd.read_csv("C:/Users/haneesha/Anaconda3/bikeRentalHourlyTrain.csv")
testset = pd.read_csv("C:/Users/haneesha/Anaconda3/bikeRentalHourlyTest.csv")

In [3]:
headers11 = ["instant0","instant","dteday","season","yr","mnth","hr","holiday","weekday","workingday","weathersit","temp","atemp","hum","windspeed","casual","registered","cnt"]
headers22 = ["instant0","instant","dteday","season","yr","mnth","hr","holiday","weekday","workingday","weathersit","temp","atemp","hum","windspeed","casual","registered","cnt"]

In [4]:
dataset.columns = headers11
testset.columns = headers22

In [5]:
dataset.head()

,instant0,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,2480,2481,2011-04-18,2,0,4,6,0,1,1,1,0.40,0.4091,0.82,0.0000,3,51,54
1,4731,4732,2011-07-21,3,0,7,1,0,4,1,2,0.74,0.7273,0.84,0.2239,5,16,21
2,7422,7423,2011-11-10,4,0,11,21,0,4,1,1,0.36,0.3333,0.57,0.2537,6,130,136
3,8588,8589,2011-12-29,1,0,12,15,0,4,1,2,0.30,0.3182,0.52,0.0896,27,150,177
4,16339,16340,2012-11-18,4,1,11,13,0,0,0,1,0.40,0.4091,0.62,0.3284,113,334,447


In [6]:
dataset.shape

(13034, 18)

In [7]:
dataset.drop('casual', axis=1, inplace=True)
dataset.drop('registered', axis=1, inplace=True)
dataset.drop('instant', axis=1, inplace=True)
dataset.drop('dteday', axis=1, inplace=True)

testset.drop('casual', axis=1, inplace=True)
testset.drop('registered', axis=1, inplace=True)
testset.drop('instant', axis=1, inplace=True)
testset.drop('dteday', axis=1, inplace=True)

In [8]:
print(dataset.describe())

           instant0        season            yr          mnth            hr  \
count  13034.000000  13034.000000  13034.000000  13034.000000  13034.000000   
mean    8694.532760      2.501151      0.503222      6.537901     11.524858   
std     5028.696269      1.108358      0.500009      3.440652      6.919784   
min        0.000000      1.000000      0.000000      1.000000      0.000000   
25%     4329.500000      2.000000      0.000000      4.000000      6.000000   
50%     8709.500000      3.000000      1.000000      7.000000     12.000000   
75%    13069.750000      3.000000      1.000000     10.000000     18.000000   
max    17378.000000      4.000000      1.000000     12.000000     23.000000   

            holiday       weekday    workingday    weathersit          temp  \
count  13034.000000  13034.000000  13034.000000  13034.000000  13034.000000   
mean       0.028464      3.006982      0.682676      1.425349      0.496873   
std        0.166301      2.006212      0.465453    

In [9]:
headers1 = ["instant0","season","yr","mnth","hr","holiday","weekday","workingday","weathersit","temp","atemp","hum","windspeed","cnt"]
headers2 = ["instant0","season","yr","mnth","hr","holiday","weekday","workingday","weathersit","temp","atemp","hum","windspeed","cnt"]

In [10]:
number = preprocessing.LabelEncoder()
for i in headers1:
    dataset[i] = number.fit_transform(dataset[i].astype("str"))

In [11]:
number = preprocessing.LabelEncoder()
for i in headers2:
    testset[i] = number.fit_transform(testset[i].astype("str"))

In [12]:
train_x, train_y = dataset[headers1[1:-1]], dataset[headers1[-1]]
test_x, test_y = testset[headers2[1:-1]], testset[headers2[-1]]

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_x)
scaler.fit(test_x)
train_x = scaler.transform(train_x)
test_x = scaler.transform(test_x)

In [14]:
from sklearn.neural_network import MLPRegressor
MLPregr = MLPRegressor(hidden_layer_sizes=(40,40,40), max_iter=100000)

In [15]:
trained_model = MLPregr.fit(train_x,train_y)

In [16]:
test_model = MLPregr.fit(test_x,test_y)

In [17]:
predictions1 = MLPregr.predict(train_x)

In [18]:
predictions2 = MLPregr.predict(test_x)

In [19]:
sum((predictions2 - test_y)**2)/len(predictions2)

35223.479530183184

In [20]:
trainn_x = pd.DataFrame(train_x)

In [21]:
trainn_y = pd.DataFrame(train_y)

In [22]:
testt_x = pd.DataFrame(test_x)

In [23]:
from sklearn.model_selection import KFold, cross_val_predict

In [24]:
kf = KFold(n_splits=5)

In [25]:
kf.get_n_splits(train_x)

5

In [26]:
MSE = 0.0
for train_index, test_index in kf.split(train_x):
    MLPregr = MLPRegressor(hidden_layer_sizes=(40,40,40,40,40))
    MLPregr.fit(trainn_x.iloc[train_index], trainn_y.iloc[train_index].values.ravel())
    predictions = MLPregr.predict(trainn_x.iloc[test_index])
    MSE += sum((predictions - trainn_y.iloc[test_index].values.ravel())**2)/len(predictions)
    
print(MSE/5.0)
    

47425.0674778


In [27]:
kf_prediction = cross_val_predict(MLPregr, trainn_x.iloc[:, 3:13 ],dataset["cnt"])
testkf_nn_pred = cross_val_predict(MLPregr, testt_x.iloc[:, 3:13 ], testset["cnt"])

In [28]:
print("Mean squared error of test set: %.2f" % mean_squared_error(testset["cnt"], testkf_nn_pred))

Mean squared error of test set: 41939.98


In [29]:
print("Mean squared error of trained set: %.2f" %mean_squared_error(dataset["cnt"], kf_prediction))

Mean squared error of trained set: 50919.69
